In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
from keras import layers

## CIFAR10 consists of 10 different types of images
#### Like airoplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck
#### 50k training images and 10k testing images 32*32 RGB pixels

In [3]:
from tensorflow.keras.datasets import cifar10

## STEP-1: Load Dataset

In [7]:
(x_train, y_train), (x_test,y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 671s 4us/step


## Step-2: convert to "float32" and normalize to 0-1 by dividing with 255
#### Float32 for better computations

In [8]:
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [9]:
x_train.shape

(50000, 32, 32, 3)

## STEP-3: Building using Sequential

In [17]:
model = keras.Sequential(
[
    keras.Input(shape=(32,32,3)),
    layers.Conv2D(32, (3,3), padding='valid', activation='relu'),  
    #32 for number of convolutional channels/ (3,3) or 3 kernal size/ 
    #padding 'valid' or 'same' valid is default and output channel size is 30
    # for same it maintain same size 32
    layers.MaxPooling2D(pool_size=(2,2)),
])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


In [21]:
model = keras.Sequential(
[
    keras.Input(shape=(32,32,3)),
    layers.Conv2D(32, (3,3), padding='valid', activation='relu'),  
    #32 for number of convolutional channels/ (3,3) or 3 kernal size/ 
    #padding 'valid' or 'same' valid is default and output channel size is 30
    # for same it maintain same size 32
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10),
    
])

## STEP-4 Model Compilation

In [22]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(learning_rate = 3e-4),
    metrics=['accuracy']
)

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)             

## Model Fitting

In [24]:
model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)

Epoch 1/10
782/782 - 56s - loss: 1.6502 - accuracy: 0.4000 - 56s/epoch - 72ms/step
Epoch 2/10
782/782 - 53s - loss: 1.3190 - accuracy: 0.5310 - 53s/epoch - 68ms/step
Epoch 3/10
782/782 - 53s - loss: 1.1815 - accuracy: 0.5823 - 53s/epoch - 68ms/step
Epoch 4/10
782/782 - 53s - loss: 1.0812 - accuracy: 0.6236 - 53s/epoch - 68ms/step
Epoch 5/10
782/782 - 53s - loss: 1.0077 - accuracy: 0.6503 - 53s/epoch - 68ms/step
Epoch 6/10
782/782 - 53s - loss: 0.9398 - accuracy: 0.6720 - 53s/epoch - 68ms/step
Epoch 7/10
782/782 - 53s - loss: 0.8893 - accuracy: 0.6910 - 53s/epoch - 68ms/step
Epoch 8/10
782/782 - 53s - loss: 0.8480 - accuracy: 0.7073 - 53s/epoch - 68ms/step
Epoch 9/10
782/782 - 53s - loss: 0.8039 - accuracy: 0.7223 - 53s/epoch - 68ms/step
Epoch 10/10
782/782 - 54s - loss: 0.7671 - accuracy: 0.7345 - 54s/epoch - 69ms/step


In [25]:
model.evaluate(x_test, y_test, batch_size = 64, verbose = 2)

157/157 - 3s - loss: 0.8561 - accuracy: 0.7060 - 3s/epoch - 21ms/step


[0.8561049103736877, 0.7059999704360962]

## Building using Functional

Batch normalization is a layer that allows every layer of the network to do learning more independently. It is used to normalize the output of the previous layers. The activations scale the input layer in normalization. Using batch normalization learning becomes efficient also it can be used as regularization to avoid overfitting of the model. The layer is added to the sequential model to standardize the input or the outputs. It can be used at several points in between the layers of the model. It is often placed just after defining the sequential model and after the convolution and pooling layers. The below code shows how to define the BatchNormalization layer for the classification of handwritten digits. We will first import the required libraries and the dataset. Use the below code for the same. 

In [31]:
def my_model():
    inputs = keras.Input(shape=(32,32,3))
    x = layers.Conv2D(32,3)(inputs) # no activation due to we use batch normalization
    x = layers.BatchNormalization()(x)                # after batch normalization we can use activation
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64,5,padding='same')(x) 
    x = layers.BatchNormalization()(x)               
    x = keras.activations.relu(x)
    x = layers.Conv2D(128,3)(x)
    x = layers.BatchNormalization()(x)               
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(10)(x)
    modelf = keras.Model(inputs=inputs, outputs = outputs)
    return modelf
    

In [32]:
model = my_model()

In [33]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 batch_normalization_3 (Batc  (None, 30, 30, 32)       128       
 hNormalization)                                                 
                                                                 
 tf.nn.relu_3 (TFOpLambda)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 15, 15, 64)        5126

In [34]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(learning_rate = 3e-4),
    metrics=['accuracy']
)

In [35]:
model.fit(x_train, y_train, batch_size = 64, epochs = 10, verbose=2)

Epoch 1/10
782/782 - 191s - loss: 1.2747 - accuracy: 0.5479 - 191s/epoch - 244ms/step
Epoch 2/10
782/782 - 194s - loss: 0.8646 - accuracy: 0.6974 - 194s/epoch - 247ms/step
Epoch 3/10
782/782 - 192s - loss: 0.7133 - accuracy: 0.7523 - 192s/epoch - 245ms/step
Epoch 4/10
782/782 - 191s - loss: 0.6085 - accuracy: 0.7882 - 191s/epoch - 245ms/step
Epoch 5/10
782/782 - 192s - loss: 0.5195 - accuracy: 0.8222 - 192s/epoch - 246ms/step
Epoch 6/10
782/782 - 192s - loss: 0.4424 - accuracy: 0.8496 - 192s/epoch - 245ms/step
Epoch 7/10
782/782 - 191s - loss: 0.3666 - accuracy: 0.8766 - 191s/epoch - 245ms/step
Epoch 8/10
782/782 - 191s - loss: 0.3152 - accuracy: 0.8931 - 191s/epoch - 244ms/step
Epoch 9/10
782/782 - 192s - loss: 0.2483 - accuracy: 0.9192 - 192s/epoch - 246ms/step
Epoch 10/10
782/782 - 192s - loss: 0.1984 - accuracy: 0.9373 - 192s/epoch - 245ms/step


In [36]:
model.evaluate(x_test, y_test, batch_size = 64, verbose = 2)

157/157 - 11s - loss: 0.9856 - accuracy: 0.7234 - 11s/epoch - 67ms/step


[0.9856141805648804, 0.7233999967575073]